# Modelo estatístico de previsão de chuva

**Sala: 2A**

**Grupo:**
> Breno Marti

> Diogo Cintra

> João Pedro Chacon Ruiz

> Luiz Felipe Valente

___
## • Imports

Importando as devidas funções para que se possa fazer a análise e filtragem dos dados, bem como a implementação do modelo de regrassão multipla.

In [ ]:
#Importando as bibliotecas
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import os
from numpy import arange
from numpy import percentile
import numpy as np
from pandas import DataFrame
from scipy.stats import probplot
import statsmodels.api as sm

## • Início da análise exploratória

Antes que se possa partir para a emplementação de qualquer modelo de previsão é necessário, primeiramente,  que se entenda como os dados se comportam e, caso preciso, manipulá-los, de forma a facilitar a construção do modelo, mas sem comprometer seus conteúdos.

In [ ]:
dados1 = pd.read_excel('macapa.xlsx')
dados1.head();

In [ ]:
datas = dados1.data
horas = dados1.hora

In [ ]:
concat = []
for e in range(len(datas)):
    #concat.append(datas[e] + ',' + horas[e])
    d = datas[e]
    d = d.replace(hour=horas[e], minute=0, second=0)
    concat.append(d)
    
#print(concat)
dia_hora = pd.DataFrame(concat)

Rodar apenas uma vez a o código abaixo

In [ ]:
dados1.insert(0,'time', dia_hora, True);
dados_2 = dados1.drop(['data', 'hora'], axis=1)
dados2 = dados1.drop(['data', 'hora'], axis=1)
dados_3 = dados_2.set_index("time")
dados_4 = dados_3.dropna();
dados3 = dados2.dropna();

In [ ]:
dados3.head()

In [ ]:
dados_4.head()

## Definindo as variáveis que vamos estudar 

A partir de estudos e leitura de papeis sobre o tema, as seguintes variáveis foram, repetidamente, dadas como as mais relevantes: temperatura, umidade, pressão atmosférica e velociadade do vento. Com base nessas pesquisas, estas mesmas foram tomadas como variáveis independentes do modelo de regressão linear múltipla.

In [ ]:
dados_4.precipitacao = dados_4.precipitacao.replace('////', 0.0)

In [ ]:
dados_4 = dados_4[['temp_inst','pressao','umid_inst','precipitacao','vento_vel']].astype('float')

In [ ]:
#dados_5 = dados_4.astype(float)
dados_log = np.log(dados_4[['temp_inst','pressao','umid_inst','precipitacao','vento_vel']])

dados4 = dados3.transpose()
dados5 = dados4.loc[['time','temp_inst', 'pressao','umid_inst', 'precipitacao','vento_vel']]
dados6 = dados5.transpose()
dados6 = dados6.sort_values(by='time')
dados6 = dados6.reset_index()
del dados6['index']
d = dados6.dropna()

In [ ]:
data = []
hora = []

for coisas in d.time.astype(str):
    teste = coisas.split()
    data.append(teste[0])
    horas = teste[1]
    hora.append(horas[0:2])

In [ ]:
final = []

anterior = data[0]
a=0
day = 0
for dia in data:
    if a < 3167:
        if data[a+1] == anterior:
            final.append(day)
        else:
            final.append(day)
            day += 1
            anterior = data[a+1]
    a += 1
final.append(250)

In [ ]:
hora = pd.Series(hora).astype(int) 
final = pd.Series(final).astype(int)

In [ ]:
d['horas'] = hora
d['dias'] = final

## Criando as colunas de seno e de cosseno tanto das horas quanto dos dias para dar ao modelo a ideia de que são variáveis cíclicas

Estas variáveis foram criadas para representar a sazonalidade nos períodos de chuva, a fim de aumentar a precisão do modelo, que agora será capaz de distinguir as diferentes estações do ano.

In [ ]:
d['hora_seno'] = np.sin(2*np.pi*hora/24)
d['hora_cosseno'] = np.cos(2*np.pi*hora/24)

In [ ]:
d['dia_seno'] = np.sin(2*np.pi*final/365.25)
d['dia_cosseno'] = np.cos(2*np.pi*final/365.25)
d.head()

___
## Analisando os dias que tiveram chuva

### Análise de correlação entre variáveis independentes

Uma grande correlação entre variáveis independentes comprometem a análise por serem redundantes entre sí, impedindo que o algoritmo atribua valores relevantes às mesmas variáveis

#### Scatter Plots com Log

In [ ]:
dados_log_com_chuva = dados_log[dados_log['precipitacao']>0]
dados_log_com_chuva.head();

In [ ]:
temperatura1_log = dados_log_com_chuva['temp_inst']
pressao1_log = dados_log_com_chuva['pressao']
umidade1_log = dados_log_com_chuva['umid_inst']
precipitacao1_log = dados_log_com_chuva['precipitacao']

In [ ]:
fig, axs = plt.subplots(1, 3)
fig.set_size_inches(20, 5) 
axs[0].scatter(temperatura1_log,pressao1_log, alpha=0.4);
axs[0].set_title('Temperatura x Pressao');
axs[1].scatter(temperatura1_log,umidade1_log, alpha=0.4);
axs[1].set_title('Temperatura x Umidade');
axs[2].scatter(pressao1_log,umidade1_log, alpha=0.4);
axs[2].set_title('Pressao x Umidade');

### Análise sem log

Novamente, se trata de uma análise a fim de selecionar variáveis independentes que de fato contribuam com o modelo

#### Scatter Plots sem Log

In [ ]:
dados7 = dados_4.reset_index()
dados_com_chuva = dados7[dados7['precipitacao'].astype(float)>0]
dados_com_chuva.head();

In [ ]:
tempo1 = dados_com_chuva['time']
temperatura1 = dados_com_chuva['temp_inst']
pressao1 = dados_com_chuva['pressao']
umidade1 = dados_com_chuva['umid_inst']
precipitacao1 = dados_com_chuva['precipitacao']

temperatura1 = temperatura1.astype(float)
umidade1 = umidade1.astype(float)
pressao1 = pressao1.astype(float)
precipitacao1 = precipitacao1.astype(float)

temperatura1_log = temperatura1_log.astype(float)
umidade1_log = umidade1_log.astype(float)
pressao1_log = pressao1_log.astype(float)
precipitacao1_log = precipitacao1_log.astype(float)

In [ ]:
fig, axs = plt.subplots(1, 3)
fig.set_size_inches(25, 7) 
axs[0].scatter(temperatura1,pressao1, alpha=0.3);
axs[0].set_title('Temperatura x Pressao');
axs[1].scatter(temperatura1,umidade1, alpha=0.3);
axs[1].set_title('Temperatura x Umidade');
axs[2].scatter(pressao1,umidade1, alpha=0.3);
axs[2].set_title('Pressao x Umidade');

### Correlação entre as variáveis independentes

Cálculo do valor númerico das correlações entre as variáveis independentes

> Sem uso de log

In [ ]:
print('Correlação entre temperatura e pressão:', temperatura1.corr(pressao1))
print('Correlação entre temperatura e umidade:', temperatura1.corr(umidade1))
print('Correlação entre pressão e umidade:',pressao1.corr(umidade1))

> Com uso de log

In [ ]:
print('Correlação entre temperatura e pressão:', np.log10(temperatura1).corr(pressao1))
print('Correlação entre temperatura e umidade:', np.log10(temperatura1).corr(umidade1))
print('Correlação entre pressão e umidade:',np.log10(pressao1).corr(umidade1))

O uso de logaritmos nos valores das variáveis ajuda, em certos casos, a mostrar correlações que antes não eram percebidas dado a sua natureza

### Correlação entre as variáveis independentes e a variável dependente

Este cálculo é crucial para que se perceba quais variáveis podem, de fato contribuir para o modelo. Essa possível contribuição depende da correlação entre as variáveis independentes escolhidas e a variável dependente a ser calculada

**Scatter plot das variáveis independentes x dependente com logaritmo, a fim de visualizar possíveis correlações**

In [ ]:
fig, axs = plt.subplots(1, 3)
fig.set_size_inches(25, 7) 
axs[0].scatter(temperatura1,np.log10(precipitacao1), alpha=0.3);
axs[0].set_title('Temperatura x Precipitação');
axs[1].scatter(umidade1,np.log10(precipitacao1), alpha=0.3);
axs[1].set_title('Umidade x Precipitação');
axs[2].scatter(pressao1,np.log10(precipitacao1), alpha=0.3);
axs[2].set_title('Pressão x Precipitação');

> Cálculo sem usar o logaritmo

In [ ]:
print('Correlação entre precipitação e pressão:',(precipitacao1).corr(pressao1))
print('Correlação entre precipitação e umidade:',(precipitacao1).corr(umidade1))
print('Correlação entre precipitação e umidade:',(precipitacao1).corr(temperatura1))

> Cálculo usando o logaritmo

In [ ]:
print('Correlação entre precipitação e pressão:', np.log10(precipitacao1).corr(pressao1))
print('Correlação entre precipitação e umidade:', np.log10(precipitacao1).corr(umidade1))
print('Correlação entre precipitação e umidade:',np.log10(precipitacao1).corr(temperatura1))

## Plot de tabelas OLS

**As tabelas OLS são usadas para que se possa ter uma noção de como os dados se comportam e como se dão cetos parâmetros relevântes para a análise**

**Tabela da precipitação em função da umidade com log:**

In [ ]:
model = sm.OLS(np.log10(precipitacao1),sm.add_constant(umidade1))
results = model.fit()
results.summary()

**Tabela da precipitação em função da umidade sem log:**

In [ ]:
model = sm.OLS(precipitacao1,sm.add_constant(umidade1))
results = model.fit()
results.summary()

**Tabela da precipitação em função da temperatura sem log:**

In [ ]:
model = sm.OLS(precipitacao1,sm.add_constant(temperatura1))
results = model.fit()
results.summary()

___
## Analisando os dias que não tiveram chuvas

### Análise com log

In [ ]:
dados_log_sem_chuva = dados_log[dados_log['precipitacao']==0]
dados_log_sem_chuva.head();

In [ ]:
temperatura2_log = dados_log_com_chuva['temp_inst']
pressao2_log = dados_log_com_chuva['pressao']
umidade2_log = dados_log_com_chuva['umid_inst']
precipitacao2_log = dados_log_com_chuva['precipitacao']

In [ ]:
fig, axs = plt.subplots(1, 3)
fig.set_size_inches(20, 5) 
axs[0].scatter(temperatura2_log,pressao2_log, alpha=0.3);
axs[0].set_title('Temperatura x Pressao');
axs[1].scatter(temperatura2_log,umidade2_log, alpha=0.3);
axs[1].set_title('Temperatura x Umidade');
axs[2].scatter(pressao2_log,umidade2_log, alpha=0.3);
axs[2].set_title('Pressao x Umidade');

### Análise sem log

In [ ]:
dados_sem_chuva = dados7[dados7['precipitacao']==0]
dados_sem_chuva.head();

In [ ]:
tempo2 = dados_sem_chuva['time']
temperatura2 = dados_sem_chuva['temp_inst']
pressao2 = dados_sem_chuva['pressao']
umidade2 = dados_sem_chuva['umid_inst']
precipitacao2 = dados_sem_chuva['precipitacao']

In [ ]:
fig, axs = plt.subplots(1, 3)
fig.set_size_inches(20, 5) 
axs[0].scatter(temperatura2,pressao2, alpha=0.2);
axs[0].set_title('Temperatura x Pressão');
axs[1].scatter(temperatura2,umidade2, alpha=0.2);
axs[1].set_title('Temperatura x Umidade');
axs[2].scatter(pressao2,umidade2, alpha=0.2);
axs[2].set_title('Pressão x Umidade');

**Valores de correlação entre as variáveis independentes**

In [ ]:
temperatura2 = temperatura2.astype(float)
umidade2 = umidade2.astype(float)
pressao2 = pressao2.astype(float)
print('Correlação entre temperatura e pressão:', temperatura2.corr(pressao2))
print('Correlação entre temperatura e umidade:', temperatura2.corr(umidade2))
print('Correlação entre pressão e umidade:',pressao2.corr(umidade2))

____

## Organização de DataFrames

> Estes são feitos a fim de possibilitar a análise do evento a partir dos dados advindos das horas que o precedem e do instante em sí

In [ ]:
dados6 = d.set_index('time')
dados7 = dados6[['temp_inst','pressao','umid_inst','precipitacao','vento_vel']]

In [ ]:
dados7.precipitacao = dados7.precipitacao.replace('////', 0.0)

In [ ]:
D = dados7.astype(float)
D.head()

In [ ]:
t = [] #hora de chuva
t1 = [] #uma hora anterior
t2 = [] #duas horas anteriores
t3 = [] #tres horas anteriores
t4 = [] #quatro horas anteriores

for e in range(len(D)):
    
    if D['precipitacao'][e]>0 and D['precipitacao'][e-1]==0 and D['precipitacao'][e-2]==0 and D['precipitacao'][e-3]==0 and D['precipitacao'][e-4]==0:
        t.append(D.iloc[e])
        t1.append(D.iloc[e-1])
        t2.append(D.iloc[e-2])
        t3.append(D.iloc[e-3])
        t4.append(D.iloc[e-4])

# Guardando as listas como DataFrames
T = pd.DataFrame(t)
T1 = pd.DataFrame(t1)
T2 = pd.DataFrame(t2)
T3 = pd.DataFrame(t3)
T4 = pd.DataFrame(t4)

# Renomeando as colunas de todos estes DataFrames -  0 Horas
T.rename(columns={'temp_inst':'temperatura_t'}, inplace=True)
T.rename(columns={'pressao':'pressao_t'}, inplace=True)
T.rename(columns={'umid_inst':'umidade_t'}, inplace=True)
T.rename(columns={'precipitacao':'precipitacao_t'}, inplace=True)
T.rename(columns={'vento_vel':'vento_t'}, inplace=True)
T[['temperatura_t', 'pressao_t', 'umidade_t','precipitacao_t','vento_t']].astype(float);

# Renomeando as colunas de todos estes DataFrames -  1 Hora atrás
T1.rename(columns={'temp_inst':'temperatura_t1'}, inplace=True)
T1.rename(columns={'pressao':'pressao_t1'}, inplace=True)
T1.rename(columns={'umid_inst':'umidade_t1'}, inplace=True)
T1.rename(columns={'precipitacao':'precipitacao_t1'}, inplace=True)
T1.rename(columns={'vento_vel':'vento_t1'}, inplace=True)
T1[['temperatura_t1', 'pressao_t1', 'umidade_t1','precipitacao_t1','vento_t1']].astype(float);

# Renomeando as colunas de todos estes DataFrames -  2 Horas atrás
T2.rename(columns={'temp_inst':'temperatura_t2'}, inplace=True)
T2.rename(columns={'pressao':'pressao_t2'}, inplace=True)
T2.rename(columns={'umid_inst':'umidade_t2'}, inplace=True)
T2.rename(columns={'precipitacao':'precipitacao_t2'}, inplace=True)
T2.rename(columns={'vento_vel':'vento_t2'}, inplace=True)
T2[['temperatura_t2', 'pressao_t2', 'umidade_t2','precipitacao_t2','vento_t2']].astype(float);

# Renomeando as colunas de todos estes DataFrames -  3 Horas atrás
T3.rename(columns={'temp_inst':'temperatura_t3'}, inplace=True)
T3.rename(columns={'pressao':'pressao_t3'}, inplace=True)
T3.rename(columns={'umid_inst':'umidade_t3'}, inplace=True)
T3.rename(columns={'precipitacao':'precipitacao_t3'}, inplace=True)
T3.rename(columns={'vento_vel':'vento_t3'}, inplace=True)
T3[['temperatura_t3', 'pressao_t3', 'umidade_t3','precipitacao_t3','vento_t3']].astype(float);

# Renomeando as colunas de todos estes DataFrames -  4 Horas atrás
T4.rename(columns={'temp_inst':'temperatura_t4'}, inplace=True)
T4.rename(columns={'pressao':'pressao_t4'}, inplace=True)
T4.rename(columns={'umid_inst':'umidade_t4'}, inplace=True)
T4.rename(columns={'precipitacao':'precipitacao_t4'}, inplace=True)
T4.rename(columns={'vento_vel':'vento_t4'}, inplace=True)
T4[['temperatura_t4', 'pressao_t4', 'umidade_t4','precipitacao_t4','vento_t4']].astype(float);

# Observação

### É crucial explicitar que, desta forma, o modelo fará a previsão da quantidade de chuva dado os valores  passados e dado o fato de que haverá uma chuva no fim da janela analisada. Isso significa que ele não prevê a chuva em um cenário geral, mas somente nos casos em que se sabe que uma chuva irá cair ou já esteja caindo.

## Selecionando a variação das variáveis

É válido que se preste devida atenção à variação nas variáveis além de seus valores em sí. Um exemplo desta mudança na forma de análise explica como uma temperatura baixa no verâo, provavelmente causada por uma frente fria, geralmente acompanha chuva, enquanto temperaturas baixas no inverno não fornecem um indicativo tão convincente. Na mesma lógica, uma queda ou aumento na pressão significam muito mais que seus valores por se só.

In [ ]:
temp = []
umid = []
pres = []
vel = []
preci = []

for e in range(len(T)):
    
    temp.append(abs((T['temperatura_t'][e])-(T3['temperatura_t3'][e])))
    umid.append(abs((T['umidade_t'][e])-(T3['umidade_t3'][e])))
    pres.append(T3['pressao_t3'][e])
    vel.append(T3['vento_t3'][e])
    preci.append(T['precipitacao_t'][e])    
       
Temp = pd.DataFrame(temp)
Umid = pd.DataFrame(umid)
Pres = pd.DataFrame(pres)
Vel = pd.DataFrame(vel)
Preci = pd.DataFrame(preci)

Temp.rename(columns={0:'temperatura_t'}, inplace=True)
Umid.rename(columns={0:'umidade_t'}, inplace=True)
Pres.rename(columns={0:'pressao_t'}, inplace=True)
Vel.rename(columns={0:'vento_t'}, inplace=True)
Preci.rename(columns={0:'precipitacao_t'}, inplace=True)
precipita = Preci['precipitacao_t'].astype(float)

a = Temp.join(Umid)
b = Vel.join(Pres)
c = b.join(precipita)
variacao = a.join(c)
variacao.dropna();

### Correlação entre a variação das VI (variáveis independentes) e a VD (variável dependente)

In [ ]:
np.log10(variacao['precipitacao_t']).corr((variacao['umidade_t']))

In [ ]:
np.log10(variacao['precipitacao_t']).corr((variacao['temperatura_t']))

In [ ]:
np.log10(variacao['precipitacao_t']).corr((variacao['pressao_t']))

In [ ]:
np.log10(variacao['precipitacao_t']).corr((variacao['vento_t']))

 A partir das análises feitas nessas células pode-se concluir que os dados das variáveis independentes possuem uma correlação muito baixa com a variável dependente.

____
## Realizando as regressões

In [ ]:
### Função utilitária para fazer a regressão com constante adicionada
def regress(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    return results

In [ ]:
Y = variacao['precipitacao_t']
X3 = variacao[['temperatura_t', 'pressao_t', 'umidade_t','vento_t']]

In [ ]:
results1 = regress(X3.values, Y.values)
results1.summary()

In [ ]:
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X3.values, Y.values, test_size=0.2, random_state=0)

In [ ]:
results2 = regress(X_train, y_train)
results2.summary()

**Regressão usando Regressão Linear Múltipla**

In [ ]:
regressor = LinearRegression()  
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

In [ ]:
#Montando um comparador a fim de visualizar a competência do modelo
p = 0
while p < len(df['Predicted']):
    if df['Predicted'][p] < 0:
        df['Predicted'][p] = 0
    
    p += 1
    
df.head(5)

In [ ]:
lista_actual = list(df['Actual'])
lista_predicted = list(df['Predicted'])

erros = []
k = 0
while k < len(lista_actual):
    erros.append(abs(lista_actual[k] - lista_predicted[k]))
    k += 1
    
print('O erro médio é:',np.mean(erros))
print('O desvio padrão do erro é:', np.std(erros))

In [ ]:
a = np.arange(0,24,1)
plt.plot(a, df['Actual'], 'bo')
plt.plot(a, df['Predicted'], 'ro')
plt.title('Comparaçao dos valores previstos com os reais')
plt.ylabel('Número de mm de chuva')
plt.grid(True)
plt.show()

In [ ]:
df.iloc[:20,:].plot(kind='bar', color=('blue', 'red'), legend=True );
plt.title('Comparaçao dos valores previstos com os reais')
plt.ylabel('Número de mm de chuva')
plt.grid(True)

**Regressão usando Random Forrest Regressor**

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Setando 100 árvores de decisão
rf = RandomForestRegressor(n_estimators = 100, random_state = 42)

# Treinando o modelo
rf.fit(X_train, y_train);

In [ ]:
# Fazendo as predições
predictions = rf.predict(X_test)

In [ ]:
df2 = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})

In [ ]:
p = 0
while p < len(df2['Predicted']):
    if df2['Predicted'][p] < 0:
        df2['Predicted'][p] = 0
    
    p += 1

In [ ]:
df2.head(10);

In [ ]:
lista_actual2 = list(df2['Actual'])
lista_predicted2 = list(df2['Predicted'])

erros2 = []
k = 0
while k < len(lista_actual2):
    erros2.append(abs(lista_actual2[k] - lista_predicted2[k]))
    k += 1
    
print('O erro médio é:',np.mean(erros2))
print('O desvio padrão do erro é:', np.std(erros2))

In [ ]:
plt.plot(a, df2['Actual'], 'bo')
plt.plot(a, df2['Predicted'], 'ro')
plt.title('Comparaçao dos valores previstos com os reais')
plt.ylabel('Número de mm de chuva')
plt.grid(True)
plt.show()

In [ ]:
df2.iloc[:20,:].plot(kind='bar', color=('blue', 'red'), legend=True );
plt.title('Comparaçao dos valores previstos com os reais')
plt.ylabel('Número de mm de chuva')
plt.grid(True)

# Conclusões

O objetivo principal do projeto é de usar dados de tempos anteriores para prever a quantidade de chuva que cairá no futuro. Para isso, um modelo de regressão linear múltipla foi escolhido para ser usado. Porém, antes que fizéssemos qualquer tipo de previsão, foi necessário que se "limpasse" os dados, filtrando e manipulando-os de forma a fazê-los, de maneira honesta, se comportarem de um modo mais adequado para o uso na regressão.

Foi nesta fase de manipulação dos dados em que se encontrou indícios de problemas que vieram a comprometer o modelo preditivo. As correlações entre as variáveis independentes escolhidas, Temperatura, Pressão e Umidade, e a variável dependente, de precipitação, se mostraram muito baixas. Esse fato implica na  dificuldade de se prever este fenômeno, dado que os dados tidos como mais valiosos, empiricamente, apresentaram correlações muito baixas, ou seja, não descrevem com qualidade o comportamento da variável a ser analisada.

Isso tudo resultou em previsões de baixíssima precisão por parte do modelo de regressão múltipla que, na prática, não valeriam de nada. Com valores de R quadrado em torno de 0.086, as previsões passaram longe dos valores reais.

Tentativas foram feitas para que se aumentasse a correlação entre os dados e a precipitação, porém, sem sucesso. Depois de tudo isso, teve-se como hipótese que o DataSet se mostrava anormal, talvez por medições de má qualidade ou, mais provavelmente, por um comportamento mais extremo dos dados (são muito escassos os dias de chuva, o que compromete a previsão). 

Tendo tudo isto em vista, entende-se como razoável a troca de DataSet por um que tenha um comportamento mais balanceado na questão de chuva e seca. E assim foi feito nas análises que se encontram no Notebook da Análise final, de São Paulo.